Let's try to use OpenCV with CUDA



## Enabling opencv on cuda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xzf /content/drive/MyDrive/opencv_builds/opencv_cuda_build_2025-09-20.tar.gz -C /

In [ ]:
!ldconfig

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero_v2.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/l

In [ ]:
import cv2

# Check the number of CUDA-enabled GPUs

device_count = cv2.cuda.getCudaEnabledDeviceCount()

print(f"CUDA-enabled devices: {device_count}")

CUDA-enabled devices: 1


# THE CODE

Important note that it is not CUDA artitechture based right now.

In [ ]:
# Import necessary libraries for setup
from google.colab import drive
from tqdm.notebook import tqdm # Use the notebook-friendly version of tqdm
import os
import sys

# --- Mount Google Drive ---
# This will prompt you for authorization.
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# --- Install/Upgrade Libraries ---
# While most are pre-installed, it's good practice to ensure they are current.
print("\nChecking/installing libraries...")
!{sys.executable} -m pip install -q numpy pandas matplotlib opencv-python-headless pillow scipy
print("Libraries are ready.")

# --- Verify Access to Your Data ---
# IMPORTANT: Before running the next cells, ensure this path points to your RAW_DIR.
# If this command lists your .tiff files, you're ready to go.
# You might need to adjust the path depending on where "tool054gain10paperBG" is located within your "My Drive".
test_path = '/content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG' # <-- UPDATE THIS PATH IF NEEDED
print(f"\nVerifying access to: {test_path}")
try:
    file_list = os.listdir(test_path)
    print(f"Successfully accessed directory. Found {len(file_list)} files/folders.")
except FileNotFoundError:
    print(f"ERROR: Could not find the directory at the specified path. Please update 'test_path' and the 'RAW_DIR' in the config cell below.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.

Checking/installing libraries...
Libraries are ready.

Verifying access to: /content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG
Successfully accessed directory. Found 406 files/folders.


In [ ]:
import os

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# MASTER CONTROL PANEL
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# Set these to True or False to run or skip steps
RUN_STEP_1_BLUR_AND_RENAME = True
RUN_STEP_2_GENERATE_MASKS = True
RUN_STEP_3_ANALYZE_AND_PLOT = True

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# CONFIGURATION PARAMETERS
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---


# Based on your link, the folder is likely not at the root.
# I will assume it's in a path like this, PLEASE VERIFY.
BASE_PATH = '/content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause'

DATA_FOLDER_NAME = 'tool113gain10paperBG'
RAW_DIR_PATH = os.path.join(BASE_PATH, DATA_FOLDER_NAME)

CONFIG = {
    # --- Directory Paths (Now adapted for Google Drive) ---
    'RAW_DIR': RAW_DIR_PATH,
    'BLURRED_DIR': f"{RAW_DIR_PATH}_blurred",
    'FINAL_MASKS_DIR': f"{RAW_DIR_PATH}_final_masks",
    'ROI_CSV_PATH': f"{RAW_DIR_PATH}_area_vs_angle.csv",
    'ROI_PLOT_PATH': f"{RAW_DIR_PATH}_area_vs_angle_plot.svg",
    'BACKGROUND_IMAGE_PATH': os.path.join(BASE_PATH, 'paper_background.tiff'), # Assuming background is in the same base folder

    # --- Image Processing Parameters ---
    'blur_kernel': 13,
    'closing_kernel': 21,

    # -- HSV Parameters --
    'h_threshold_min': 70 // 2,
    'h_threshold_max': 100 // 2,
    's_threshold_min': 15 * 2.55,
    's_threshold_max': 70 * 2.55,
    'V_threshold_min': 45 * 2.55,
    'V_threshold_max': 55 * 2.55,

    # -- LAB Parameters --
    'L_threshold_min': 50 * 2.55,
    'L_threshold_max': 56 * 2.55,
    'a_threshold_min': -10 + 128,
    'a_threshold_max': -1 + 128,
    'b_threshold_min': -10 + 128,
    'b_threshold_max': -8 + 128,

    # --- Background Subtraction Parameters ---
    'BACKGROUND_SUBTRACTION_METHOD': 'lab', # Options: 'none', 'absdiff', 'lab'
    'APPLY_MULTICHANNEL_MASK': False,
    'DIFFERENCE_THRESHOLD': 21,

    # --- Data Analysis Parameters ---
    'images_for_366_deg': 363,
    'roi_height': 300,
    'outlier_std_dev_factor': 2.0,
    'APPLY_MOVING_AVERAGE': True,
    'MOVING_AVERAGE_WINDOW': 5,
}

print("Configuration loaded.")
print(f"Raw data is expected at: {CONFIG['RAW_DIR']}")
print(f"Blurred data will be saved to: {CONFIG['BLURRED_DIR']}")
print(f"Masks will be saved to: {CONFIG['FINAL_MASKS_DIR']}")

Configuration loaded.
Raw data is expected at: /content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG
Blurred data will be saved to: /content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG_blurred
Masks will be saved to: /content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG_final_masks


In [ ]:
from PIL import Image, ImageFilter
import cv2
import numpy as np

# --- All functions from your filters.py file ---

def create_multichannel_mask(image_object, config):
    try:
        rgb_image_np = np.array(image_object.convert('RGB'))
        lab_image_np = cv2.cvtColor(rgb_image_np, cv2.COLOR_RGB2LAB)
        hsv_image_np = cv2.cvtColor(rgb_image_np, cv2.COLOR_RGB2HSV)
        L_channel, a_channel, b_channel = cv2.split(lab_image_np)
        H_channel, S_channel, V_channel = cv2.split(hsv_image_np)
        L_mask = (L_channel >= config['L_threshold_min']) & (L_channel <= config['L_threshold_max'])
        a_mask = (a_channel >= config['a_threshold_min']) & (a_channel <= config['a_threshold_max'])
        b_mask = (b_channel >= config['b_threshold_min']) & (b_channel <= config['b_threshold_max'])
        H_mask = (H_channel >= config['h_threshold_min']) & (H_channel <= config['h_threshold_max'])
        S_mask = (S_channel >= config['s_threshold_min']) & (S_channel <= config['s_threshold_max'])
        V_mask = (V_channel >= config['V_threshold_min']) & (V_channel <= config['V_threshold_max'])
        final_boolean_mask = ~L_mask
        final_mask_visual = final_boolean_mask.astype(np.uint8) * 255
        return Image.fromarray(final_mask_visual)
    except Exception as e:
        print(f"An error occurred during multi-channel masking: {e}")
        return None

def apply_median_blur(tiff_path, kernel_size=13):
    try:
        with Image.open(tiff_path) as img:
            return img.filter(ImageFilter.MedianFilter(size=kernel_size))
    except Exception as e:
        print(f"An error occurred during blurring: {e}")
        return None

def fill_holes(binary_mask_object):
    try:
        mask = np.array(binary_mask_object.convert('L'))
        mask_floodfill = mask.copy()
        h, w = mask.shape[:2]
        bordered_mask = np.zeros((h + 2, w + 2), np.uint8)
        cv2.floodFill(mask_floodfill, bordered_mask, (0, 0), 255)
        mask_floodfill_inv = cv2.bitwise_not(mask_floodfill)
        filled_mask = (mask | mask_floodfill_inv)
        return Image.fromarray(filled_mask)
    except Exception as e:
        print(f"An error occurred during hole filling: {e}")
        return None

def morph_closing(binary_mask_object, kernel_size=5):
    try:
        mask = np.array(binary_mask_object.convert('L'))
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        return Image.fromarray(closing)
    except Exception as e:
        print(f"An error occurred during morphological closing: {e}")
        return None

def keep_largest_contour(binary_mask_object):
    try:
        mask = np.array(binary_mask_object.convert('L'))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            new_mask = np.zeros_like(mask)
            cv2.drawContours(new_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
            return Image.fromarray(new_mask)
        return binary_mask_object
    except Exception as e:
        print(f"An error occurred while finding the largest contour: {e}")
        return None

def background_subtraction_absdiff(image_object, background_np, config):
    try:
        sample_gray = cv2.cvtColor(np.array(image_object.convert('RGB')), cv2.COLOR_RGB2GRAY)
        background_gray = cv2.cvtColor(background_np, cv2.COLOR_RGB2GRAY)
        diff_image = cv2.absdiff(sample_gray, background_gray)
        _, binary_mask = cv2.threshold(diff_image, config['DIFFERENCE_THRESHOLD'], 255, cv2.THRESH_BINARY)
        return Image.fromarray(binary_mask)
    except Exception as e:
        print(f"An error occurred during absdiff background subtraction: {e}")
        return None

def background_subtraction_lab(image_object, background_np, config):
    try:
        sample_lab = cv2.cvtColor(np.array(image_object.convert('RGB')), cv2.COLOR_RGB2LAB).astype(np.float32)
        background_lab = cv2.cvtColor(background_np, cv2.COLOR_RGB2LAB).astype(np.float32)
        delta_E = np.sqrt(np.sum((sample_lab - background_lab)**2, axis=2))
        diff_image = cv2.normalize(delta_E, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        _, binary_mask = cv2.threshold(diff_image, config['DIFFERENCE_THRESHOLD'], 255, cv2.THRESH_BINARY)
        return Image.fromarray(binary_mask)
    except Exception as e:
        print(f"An error occurred during LAB background subtraction: {e}")
        return None

print("Utility functions defined.")

Utility functions defined.


In [ ]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
import os

def load_image_batch(file_paths, batch_size):
    """
    A generator function that loads images from disk in batches.
    This is memory-efficient for large datasets.
    We use cv2.imread since we will be using OpenCV's CUDA module.
    """
    for i in range(0, len(file_paths), batch_size):
        batch_paths = file_paths[i:i+batch_size]
        images = [cv2.imread(p) for p in batch_paths]
        # Filter out any images that failed to load
        loaded_images = [img for img in images if img is not None]
        if loaded_images:
            yield loaded_images, batch_paths

print("Batch loading helper function defined.")


Batch loading helper function defined.


In [ ]:
def run_step1_gpu(config, batch_size=32):
    """
    First, renames raw files (CPU task), then applies a GPU-accelerated
    Gaussian blur in batches.
    """
    raw_dir = config['RAW_DIR']
    blurred_dir = config['BLURRED_DIR']

    os.makedirs(blurred_dir, exist_ok=True)

    try:
        image_files = sorted([f for f in os.listdir(raw_dir) if f.endswith(('.tiff', '.tif'))])
        if not image_files:
            print(f"No raw images found in '{raw_dir}'. Skipping.")
            return
    except FileNotFoundError:
        print(f"Error: Raw data directory not found at '{raw_dir}'.")
        return

    # --- Part 1: Rename Raw Files (This remains a CPU task) ---
    if '_degrees.tiff' not in image_files[0]:
        print("Renaming raw files to include their rotation angle...")
        angle_step = 366.0 / config['images_for_366_deg']
        renamed_files = []
        for i, filename in enumerate(tqdm(image_files, desc="Renaming Raw Files")):
            current_angle = i * angle_step
            new_filename = f"{current_angle:07.2f}_degrees.tiff"
            old_path = os.path.join(raw_dir, filename)
            new_path = os.path.join(raw_dir, new_filename)
            try:
                os.rename(old_path, new_path)
                renamed_files.append(new_filename)
            except OSError as e:
                print(f"\nError renaming {filename}: {e}")
                continue
        image_files = sorted(renamed_files)
        print("File renaming complete.")
    else:
        print("Raw files appear to be already renamed. Skipping renaming.")

    # --- Part 2: GPU-Accelerated Blurring ---

    # Check if we can even use CUDA
    if cv2.cuda.getCudaEnabledDeviceCount() == 0:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! ERROR: No CUDA-enabled GPU found. Cannot proceed. !!!")
        print("!!! In Colab, go to 'Runtime' -> 'Change runtime type' -> 'T4 GPU' !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return

    print(f"\nFound {cv2.cuda.getCudaEnabledDeviceCount()} CUDA-enabled GPU(s).")

    if len(os.listdir(blurred_dir)) >= len(image_files):
        print(f"Blurred images already exist in '{blurred_dir}'. Skipping blur step.")
        return

    print(f"Applying GPU-accelerated Gaussian blur to {len(image_files)} images...")

    # Create the GPU filter ONCE outside the loop
    # CV_8UC3 means 8-bit unsigned integer, 3 channels (standard BGR image)
    # The kernel size must be odd.
    kernel_size = config['blur_kernel'] if config['blur_kernel'] % 2 != 0 else config['blur_kernel'] + 1
    gpu_filter = cv2.cuda.createGaussianFilter(cv2.CV_8UC3, -1, (kernel_size, kernel_size), 0)

    # Get full paths for the batch loader
    full_file_paths = [os.path.join(raw_dir, f) for f in image_files]

    # Process images in batches
    for image_batch, path_batch in tqdm(load_image_batch(full_file_paths, batch_size),
                                         total=len(full_file_paths)//batch_size, desc="Blurring Batches"):

        # 1. Create a GpuMat object for GPU memory
        gpu_frame = cv2.cuda_GpuMat()

        for i, cpu_image in enumerate(image_batch):
            try:
                # 2. Upload image from CPU RAM to GPU VRAM
                gpu_frame.upload(cpu_image)

                # 3. Apply the filter ON THE GPU
                blurred_gpu_frame = gpu_filter.apply(gpu_frame)

                # 4. Download the result from GPU VRAM back to CPU RAM
                blurred_cpu_image = blurred_gpu_frame.download()

                # 5. Save the processed image
                filename = os.path.basename(path_batch[i])
                output_path = os.path.join(blurred_dir, filename)
                cv2.imwrite(output_path, blurred_cpu_image)

            except Exception as e:
                print(f"Error processing {os.path.basename(path_batch[i])}: {e}")

print("Step 1 GPU function defined.")


Step 1 GPU function defined.


In [ ]:
def run_step2(config):
    """
    Processes pre-blurred images by combining background subtraction and
    multi-channel color masking based on config settings.
    """
    method = config.get('BACKGROUND_SUBTRACTION_METHOD', 'none').lower()
    use_mc_mask = config.get('APPLY_MULTICHANNEL_MASK', False)
    if method == 'none' and not use_mc_mask:
        print("Warning: All masking methods are disabled. Exiting.")
        return

    input_dir = config['BLURRED_DIR']
    output_dir = config['FINAL_MASKS_DIR']
    os.makedirs(output_dir, exist_ok=True)

    try:
        image_files = sorted([f for f in os.listdir(input_dir) if f.endswith(('.tiff', '.tif'))])
    except FileNotFoundError:
        print(f"Error: Blurred data directory not found at '{input_dir}'.")
        return

    background_image_np = None
    if method in ['absdiff', 'lab']:
        try:
            bg_path = config['BACKGROUND_IMAGE_PATH']
            background_image_np = np.array(Image.open(bg_path))
            print(f"Using '{method}' method with background image: {bg_path}")
        except FileNotFoundError:
            print(f"Warning: Background image not found for '{method}'. Disabling subtraction.")
            method = 'none'

    print(f"Generating final masks for {len(image_files)} images...")

    for filename in tqdm(image_files, desc="Generating Masks"):
        image_path = os.path.join(input_dir, filename)
        try:
            blurred_image = Image.open(image_path)
            bg_mask_np, color_mask_np = None, None

            if method == 'absdiff' and background_image_np is not None:
                bg_mask_pil = background_subtraction_absdiff(blurred_image, background_image_np, config)
                if bg_mask_pil: bg_mask_np = np.array(bg_mask_pil)
            elif method == 'lab' and background_image_np is not None:
                bg_mask_pil = background_subtraction_lab(blurred_image, background_image_np, config)
                if bg_mask_pil: bg_mask_np = np.array(bg_mask_pil)

            if use_mc_mask:
                color_mask_pil = create_multichannel_mask(blurred_image, config)
                if color_mask_pil: color_mask_np = np.array(color_mask_pil)

            if bg_mask_np is not None and color_mask_np is not None:
                initial_mask_np = cv2.bitwise_or(bg_mask_np, color_mask_np)
            elif bg_mask_np is not None: initial_mask_np = bg_mask_np
            elif color_mask_np is not None: initial_mask_np = color_mask_np
            else: continue

            initial_mask = Image.fromarray(initial_mask_np)
            filled1 = fill_holes(initial_mask)
            closed = morph_closing(filled1, kernel_size=config['closing_kernel'])
            largest_contour = keep_largest_contour(closed)
            final_mask = fill_holes(largest_contour)

            if final_mask:
                output_path = os.path.join(output_dir, filename)
                final_mask.save(output_path, 'TIFF')
        except Exception as e:
            print(f"Failed to process {filename}. Error: {e}")

print("Step 2 function defined.")

Step 2 function defined.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import convolve1d

def find_roi_and_calculate_area(mask_np, roi_height):
    white_pixel_coords = np.where(mask_np == 255)
    if white_pixel_coords[0].size == 0: return 0
    last_row = white_pixel_coords[0].max()
    first_row = max(0, last_row - roi_height)
    roi = mask_np[first_row:last_row, :]
    return np.sum(roi) / 255

def run_step3(config):
    """
    Analyzes final masks within an ROI, saves data to CSV, filters, and plots.
    """
    input_dir = config['FINAL_MASKS_DIR']

    try:
        image_files = sorted([f for f in os.listdir(input_dir) if f.endswith(('.tiff', '.tif'))])
        if not image_files:
            print(f"No final masks found in '{input_dir}'. Skipping.")
            return
    except FileNotFoundError:
        print(f"Error: Final masks directory not found at '{input_dir}'.")
        return

    results = []
    print(f"Starting ROI analysis for {len(image_files)} masks...")
    for filename in tqdm(image_files, desc="Analyzing ROI"):
        try:
            angle = float(filename.split('_')[0])
            image_path = os.path.join(input_dir, filename)
            mask_np = np.array(Image.open(image_path))
            roi_area = find_roi_and_calculate_area(mask_np, config['roi_height'])
            results.append({'Angle (Degrees)': angle, 'ROI Area (Pixels)': roi_area})
        except (ValueError, IndexError):
            print(f"Could not parse angle from filename: {filename}. Skipping.")
            continue

    if not results:
        print("No data was generated from ROI analysis.")
        return

    df = pd.DataFrame(results)

    target_column = 'ROI Area (Pixels)'
    if config.get('APPLY_MOVING_AVERAGE', False):
        window_size = config.get('MOVING_AVERAGE_WINDOW', 5)
        print(f"Applying wrap-around moving average with window size {window_size}...")
        weights = np.ones(window_size) / window_size
        smoothed_data = convolve1d(df['ROI Area (Pixels)'], weights=weights, mode='wrap')
        df['Smoothed ROI Area'] = smoothed_data
        target_column = 'Smoothed ROI Area'

    csv_path = config['ROI_CSV_PATH']
    csv_dir = os.path.dirname(csv_path)
    if csv_dir: os.makedirs(csv_dir, exist_ok=True)
    df.to_csv(csv_path, index=False)
    print(f"ROI data saved to '{csv_path}'")

    mean, std = df[target_column].mean(), df[target_column].std()
    factor = config['outlier_std_dev_factor']
    inliers = df[(df[target_column] >= mean - factor * std) & (df[target_column] <= mean + factor * std)]
    print(f"Removed {len(df) - len(inliers)} outliers from '{target_column}'.")

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))

    if target_column == 'Smoothed ROI Area':
        ax.scatter(inliers['Angle (Degrees)'], inliers['ROI Area (Pixels)'], color='lightgray', s=10, label='Raw Data')

    ax.plot(inliers['Angle (Degrees)'], inliers[target_column], marker='.', linestyle='-', markersize=4, label='Smoothed Data')
    ax.set_title('Tool ROI Area vs. Rotation Angle', fontsize=18, fontweight='bold')
    ax.set_xlabel('Angle (Degrees)', fontsize=14)
    ax.set_ylabel('Projected Area in ROI (Pixel Count)', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(True)
    ax.legend()
    ax.set_xlim(0, 360)
    ax.set_xticks(np.arange(0, 361, 30))
    plt.tight_layout()

    plot_path = config['ROI_PLOT_PATH']
    plot_dir = os.path.dirname(plot_path)
    if plot_dir: os.makedirs(plot_dir, exist_ok=True)

    plt.savefig(plot_path, format='svg', dpi=300)
    print(f"Plot saved in SVG format to '{plot_path}'")

    plt.show()

print("Step 3 function defined.")

Step 3 function defined.


In [ ]:
def main():
    """
    Orchestrates the entire image processing and analysis pipeline.
    """
    if RUN_STEP_1_BLUR_AND_RENAME:
        print("\n--- Running Step 1: Blur and Rename (GPU Accelerated) ---")
        # Call the new GPU function
        run_step1_gpu(CONFIG, batch_size=64) # You can tune the batch_size
        print("--- Step 1 Complete ---")

    if RUN_STEP_2_GENERATE_MASKS:
        print("\n--- Running Step 2: Generate Final Masks ---")
        run_step2(CONFIG)
        print("--- Step 2 Complete ---")

    if RUN_STEP_3_ANALYZE_AND_PLOT:
        print("\n--- Running Step 3: Analyze ROI and Plot ---")
        run_step3(CONFIG)
        print("--- Step 3 Complete ---")

    print("\nPipeline finished.")

if __name__ == "__main__":
    main()



--- Running Step 1: Blur and Rename (GPU Accelerated) ---
Raw files appear to be already renamed. Skipping renaming.

Found 1 CUDA-enabled GPU(s).
Applying GPU-accelerated Gaussian blur to 406 images...


Blurring Batches:   0%|          | 0/6 [00:00<?, ?it/s]

--- Step 1 Complete ---

--- Running Step 2: Generate Final Masks ---
Generating final masks for 406 images...


Generating Masks:   0%|          | 0/406 [00:00<?, ?it/s]

--- Step 2 Complete ---

--- Running Step 3: Analyze ROI and Plot ---
No final masks found in '/content/drive/MyDrive/Before Cleaning/PhD_Projects/tool_monitoring/2edge_tool_clause/tool113gain10paperBG_final_masks'. Skipping.
--- Step 3 Complete ---

Pipeline finished.


## CUDA Architecture suggested by GEMINI
https://docs.google.com/document/d/15d02dWwPpJlCkN6oi-Ei68793BBVqM7K6ulhK00RwVo/edit?tab=t.0
